In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import * 
pd.options.display.max_columns = 1000

import tensorflow as tf
import keras
from keras.layers import * 

%matplotlib inline

In [5]:
df_train = pd.read_csv("/data/MNIST/mnist_train.csv", header=None)
df_test = pd.read_csv("/data/MNIST/mnist_test.csv", header=None)

X_train = (df_train.iloc[:, 1:].values/255).reshape(-1, 28, 28, 1) # width, height, channel
X_test = (df_test.iloc[:, 1:].values/255).reshape(-1, 28, 28, 1)
y_train = df_train.iloc[:, 0].values
y_test = df_test.iloc[:, 0].values

In [7]:
X_train.shape, X_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [13]:
n_x = 28 * 28
n_y = 10
tf.set_random_seed(1)

def build_model():
    model = keras.Sequential()
    model.add(Conv2D(filters=32, activation="relu", kernel_size=(5, 5), input_shape = (28, 28, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=64, activation="relu", kernel_size=(3, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dropout(rate=0.8))
    model.add(Dense(activation="relu", units=100))
    model.add(Dense(activation="softmax", units=n_y))
    
    model.compile(loss=keras.losses.categorical_crossentropy
                  , optimizer="adam"
                  , metrics = ["accuracy"])
    return model

model = build_model()
model.summary()

model.fit(x = X_train
          , y = keras.utils.to_categorical(y_train)
          , verbose = 1
          , epochs = 10
          , batch_size= 64)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 10, 10, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 1600)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               160100    
__________

In [14]:
model.evaluate(X_test, keras.utils.to_categorical(y_test) )

10000/10000 [==============================] - 4s 382us/step


[0.019832993313824408, 0.9941]

In [11]:
(5 * 5 * 1  + 1) * 32 

832

In [12]:
(3 * 3 * 32 + 1) * 64

18496

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 400)               314000    
_________________________________________________________________
dense_4 (Dense)              (None, 10)                4010      
Total params: 318,010
Trainable params: 318,010
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.evaluate(x=X_test, y = keras.utils.to_categorical(y_test))

10000/10000 [==============================] - 1s 66us/step


[0.06574365932492365, 0.9821]

In [9]:
keras.utils.to_categorical(y_train).shape

(60000, 10)

In [14]:
tf.__version__

'1.5.0'

In [16]:
y_test_pred = model.predict(X_test)

In [17]:
y_test_pred.shape

(10000, 10)

In [19]:
pd.DataFrame(y_test_pred).head()

,0,1,2,3,4,5,6,7,8,9
0,3.796711e-12,2.652959e-12,2.059882e-08,3.307889e-06,3.272547e-18,9.282589e-11,2.933753e-16,9.999967e-01,1.209711e-10,4.592330e-08
1,3.231471e-11,5.319081e-08,1.000000e+00,1.486767e-09,2.832140e-19,1.690367e-12,2.508506e-11,7.602307e-19,1.902119e-09,5.331110e-18
2,2.045858e-08,9.991222e-01,3.435578e-04,4.912526e-08,4.961033e-06,6.443893e-07,1.626363e-06,1.801484e-04,3.468152e-04,1.747640e-08
3,9.999980e-01,1.811872e-11,1.861081e-06,2.689628e-11,3.700573e-13,6.596430e-10,1.625927e-07,6.357512e-09,1.004129e-13,2.307705e-08
4,1.108941e-10,8.840528e-13,7.023003e-09,1.641068e-12,9.999671e-01,4.455092e-13,7.003552e-09,2.096153e-08,2.777511e-10,3.294671e-05


In [21]:
model.save("mnist.h5")

In [22]:
model = keras.models.load_model("mnist.h5")

In [24]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 400)               314000    
_________________________________________________________________
dense_4 (Dense)              (None, 10)                4010      
Total params: 318,010
Trainable params: 318,010
Non-trainable params: 0
_________________________________________________________________


In [20]:
np.argmax(y_test_pred, axis=1)

array([7, 2, 1, ..., 4, 5, 6])